In [ ]:
import numpy as np
import pandas as pd
from getdist import plots
from getdist.gaussian_mixtures import GaussianND

# 1. Data manipulation from cosmosis output and post-process to better visualization


In [ ]:
txt_string = "./numcosmo_fisher.txt"
f = open(txt_string, "r")

for line in f:
    temp = line.split()
    # print(temp)

# print(temp)


fisher_lsst_3x2pt = np.loadtxt(txt_string, dtype="S0")
corr = []
means = []
stddevs = []
names = []
for line in fisher_lsst_3x2pt:
    name = str(line[0]).split("[")[0]
    names.append(name[2:])
    temp_arr = []
    for elem in line[1:]:
        try:
            temp_arr.append(float(elem))
        except:
            pass
        # print(elem)

    mean = temp_arr[0]
    stddev = temp_arr[1]
    temp_arr = temp_arr[2:]
    means.append(mean)
    stddevs.append(stddev)
    corr.append(temp_arr)
    # print(name, temp_arr)
# print(fisher_lsst_3x2pt[0])
# print(corr)

for i in range(len(corr)):
    # corr[i][i]*=stddevs[i]*stddevs[i]
    for j in range(len(corr[i])):
        corr[i][j] *= (stddevs[i] * stddevs[j]) / 2

cov = corr
invcov = np.linalg.inv(cov)
print(invcov)

# 2. Ploting the fisher contours using getdist


In [ ]:
# Constructing a GaussianND object using fiducial values and the inverse Fisher matrix
lsst_3x2pt_fisher_contour = GaussianND(means, invcov, names=names)

# Initialize a subplot plotter for generating the plot
g = plots.get_subplot_plotter()

# Customize the plot settings
g.settings.figure_legend_frame = True
g.settings.axes_labelsize = 15
g.settings.legend_fontsize = 30

# Generate the triangle plot
g.triangle_plot(
    [lsst_3x2pt_fisher_contour],
    filled=True,
    legend_labels=["LSSTY1 3x2pt"],
    contour_colors=["blue"],
)

# Optional: Add parameter markers to the plot
# g.add_param_markers(lsst_3x2pt_vary_param_fid_values, color='black', ls='--')

In [ ]:
print(lsst_3x2pt_fisher_df)
print(lsst_3x2pt_fisher_df_inv)

In [ ]:
# Defining input file paths
txt_string = "./forecast_3x2pt_PT_samples.txt"
means_string = "../means.txt"


"""---- Data manipulation from cosmosis output to create more organized pandas DataFrames ----"""
# Loading the Fisher matrix from the text file
fisher_lsst_3x2pt = np.loadtxt(txt_string)

# Reading the first line of the file to extract parameter names
with open(txt_string, "r") as file:
    first_line = file.readline().strip()

# Extracting parameter names, assuming they are space-separated and may contain "--"
param_names = first_line.split()
param_names = [param.split("--")[-1] for param in param_names]

# Creating a DataFrame for the Fisher matrix with parameters as rows and columns
lsst_3x2pt_fisher_df = pd.DataFrame(
    fisher_lsst_3x2pt, columns=param_names, index=param_names
)

# Creating a DataFrame to store 'mean' and 'sigma' values
lsst_3x2pt_means_df = pd.DataFrame(index=["mean", "sigma"])

# Reading the 'means' file and extracting mean and error data
with open(means_string, "r") as file:
    next(file)  # Skipping the first line, if necessary
    for line in file:
        lines = line.split()  # Assuming values are space-separated
        param_line = [param.split("--")[-1] for param in lines]  # Handling the "--"
        key = param_line[0]  # Parameter name
        mean = float(param_line[1])  # Mean value
        sigma = float(param_line[2])  # Error
        lsst_3x2pt_means_df[key] = [mean, sigma]  # Adding to DataFrame
"""----------------------------------------------------------------------------------------------"""


# Transposing the DataFrame for easier access to values by parameter
lsst_3x2pt_means_df = lsst_3x2pt_means_df.T

# ---- Converting pandas DataFrames to numpy arrays ---- #

# List of varying parameters
lsst_3x2pt_vary_param = lsst_3x2pt_means_df.index

# Fiducial values (mean)
lsst_3x2pt_vary_param_fid_values = lsst_3x2pt_means_df["mean"]

# Errors (sigma)
lsst_3x2pt_vary_param_fid_errors = lsst_3x2pt_means_df["sigma"]

# Inverting the Fisher matrix
lsst_3x2pt_fisher_df_inv = np.linalg.inv(lsst_3x2pt_fisher_df)